<a href="https://colab.research.google.com/github/arkosarker07/Political-Biasness-identifier/blob/main/BiasDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AI4Bharat/IndicTrans2.git
%cd /content/IndicTrans2/huggingface_interface
!pip install -r requirements.txt
!pip install sentencepiece bitsandbytes scipy accelerate
!git clone https://github.com/VarunGumma/IndicTransToolkit.git
%cd IndicTransToolkit
!pip install --editable ./
%cd ..

Cloning into 'IndicTrans2'...
remote: Enumerating objects: 767, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 767 (delta 151), reused 105 (delta 104), pack-reused 585 (from 3)
Receiving objects: 100% (767/767), 4.16 MiB | 19.80 MiB/s, done.
Resolving deltas: 100% (493/493), done.
/content/IndicTrans2/huggingface_interface
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/content/IndicTrans2/huggingface_interface


In [1]:
import torch
import re
import gradio as gr
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig
from IndicTransToolkit.processor import IndicProcessor

# Setup device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 2  # Reduced for memory safety
src_lang, tgt_lang = "ben_Beng", "eng_Latn"

# Initialize model + tokenizer with 8-bit quantization
def initialize_model_and_tokenizer(ckpt_dir, quantization="8-bit"):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig is None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()
    return tokenizer, model

# Smart chunking
def split_into_chunks(text, max_chars=300):
    sentences = re.split(r'(?<=[।!?])\s+', text.strip())
    chunks = []
    current = ""
    for sentence in sentences:
        if len(current) + len(sentence) <= max_chars:
            current += " " + sentence
        else:
            if current:
                chunks.append(current.strip())
            current = sentence
    if current:
        chunks.append(current.strip())
    return chunks

# Batch translation
def batch_translate(input_sentences, model, tokenizer, ip, mode="Fast"):
    translations = []
    beam_size = 3 if mode == "Fast" else 5
    max_length = 200 if mode == "Fast" else 256

    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i: i + BATCH_SIZE]
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        try:
            with torch.no_grad():
                generated_tokens = model.generate(
                    **inputs,
                    use_cache=True,
                    min_length=0,
                    max_length=max_length,
                    num_beams=beam_size,
                    num_return_sequences=1,
                )

            generated_tokens = tokenizer.batch_decode(
                generated_tokens,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

            translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        except Exception as e:
            translations += ["[Translation Failed]"] * len(batch)

        del inputs
        torch.cuda.empty_cache()

    return translations

# Load model with 8-bit quantization
quantization = "8-bit"
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, quantization)
ip = IndicProcessor(inference=True)

# Bias classification model
bias_pipe = pipeline(
    "text-classification",
    model="matous-volf/political-leaning-deberta-large",
    tokenizer="microsoft/deberta-v3-large",
)

label_map = {
    "LABEL_0": "Left",
    "LABEL_1": "Center",
    "LABEL_2": "Right"
}

# Main function
def translate_and_classify(bn_text, mode):
    if not bn_text.strip():
        return "Please enter Bangla news content.", "", ""

    bn_sents = split_into_chunks(bn_text.strip(), max_chars=300)
    translations = batch_translate(bn_sents, indic_en_model, indic_en_tokenizer, ip, mode)
    english_text = " ".join(translations)

    if "[Translation Failed]" in translations:
        return "Translation failed for some parts.", "", english_text

    result = bias_pipe(english_text)[0]
    label = label_map.get(result["label"], result["label"])
    score = result["score"]

    return f"{label}", f"{score:.2%}", english_text

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## Bangla & English News Bias Detector")

    with gr.Row():
        with gr.Column():
            bangla_input = gr.Textbox(label="Paste News Article", lines=10, placeholder="Here...")
            mode_selector = gr.Radio(["Fast", "Accurate"], value="Fast", label="Translation Mode")
            submit_btn = gr.Button("Analyze Bias")

        with gr.Column():
            bias_output = gr.Textbox(label="Bias", interactive=False)
            score_output = gr.Textbox(label="Confidence", interactive=False)
            eng_translation = gr.Textbox(label="English Translation", lines=10, interactive=False)

    submit_btn.click(translate_and_classify, inputs=[bangla_input, mode_selector], outputs=[bias_output, score_output, eng_translation])

demo.launch()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenization_indictrans.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


dict.SRC.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://72bc24694e0d8cd393.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
